In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap= 50
)

loader = UnstructuredFileLoader("./files/something.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorestores = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorestores.as_retriever()

final_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        I have found the following information about Victory Mansions:
        ----
        {context}
        """
     ),
    ("human", "{question}")
])

map_doc_prompt = ChatPromptTemplate.format_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            {context}
            """
        )
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return  "\n\n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question": question
        }).content for doc in documents
    )
    
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

chain = { "context" : map_chain ,"question": RunnablePassthrough() } | final_prompt | llm

chain.invoke("Describe Victory Mansions")
